In [8]:
from __future__ import print_function
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import log_loss
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import xgboost as xgb


def load_train():
    train = pd.read_csv('./train.csv')
    labels = train.target.values
    lbl_enc = preprocessing.LabelEncoder()
    labels = lbl_enc.fit_transform(labels)
    train = train.drop('id', axis=1)
    train = train.drop('target', axis=1)
    return np.log(train.values + 1), labels.astype('int32')


def load_test():
    test = pd.read_csv('./test.csv')
    test = test.drop('id', axis=1)
    return test.values


def write_submission(preds, output):
    sample = pd.read_csv('./data/sampleSubmission.csv')
    preds = pd.DataFrame(
        preds, index=sample.id.values, columns=sample.columns[1:])
    preds.to_csv(output, index_label='id')

In [16]:
def score(params):
    print ("Training with params : ")
    print (params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round,watchlist,early_stopping_rounds=10)
    predictions = model.predict(dvalid).reshape((X_val.shape[0], 9))
    score = log_loss(y_val, predictions)
    print ("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


def optimize(trials):
    space = {
             'n_estimators' : 300,
             #'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'eta' : 0.1,
             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             #'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'subsample' : 0.8,
             #'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.4, 0.9, 0.1),
             'num_class' : 9,
             'eval_metric': 'mlogloss',
             'objective': 'multi:softprob',
             'nthread' : 4,
             'silent' : 1
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=20)


In [24]:
trials = Trials()

X, y = load_train()
split = StratifiedShuffleSplit(y,n_iter=1,test_size=0.2)
for train_idx,val_idx in split:
    pass
X_train = X[train_idx]
y_train = y[train_idx]
X_val = X[val_idx]
y_val = y[val_idx]

In [25]:
optimize(trials)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:2.001784	eval-mlogloss:2.003501
[1]	train-mlogloss:1.846081	eval-mlogloss:1.849742
[2]	train-mlogloss:1.724411	eval-mlogloss:1.729293
[3]	train-mlogloss:1.623380	eval-mlogloss:1.629686
[4]	train-mlogloss:1.549878	eval-mlogloss:1.557979
[5]	train-mlogloss:1.474725	eval-mlogloss:1.483581
[6]	train-mlogloss:1.411352	eval-mlogloss:1.421413
[7]	train-mlogloss:1.352709	eval-mlogloss:1.364212
[8]	train-mlogloss:1.299925	eval-mlogloss:1.312753
[9]	train-mlogloss:1.255069	eval-mlogloss:1.269052
[10]	train-mlogloss:1.214729	eval-mlogloss:1.229378
[11]	train-mlogloss:1.174989	eval-mlogloss:1.190063
[12]	train-mlogloss:1.138153	eval-mlogloss:1.154024
[13]	train-mlogloss:1.107674	eval-mlogloss:1.124754
[14]	train-mlogloss:1.078863	eval-mlogloss:1.096538
[15]	train-mlogloss:1.050328	eval-mlogloss:1.068710
[16]	train-mlogloss:1.026186	eval-mlogloss:1.045070
[17]	train-mlogloss:1.001831	eval-mlogloss:1.021575
[18]	train-mlog

Training with params : 
{'n_estimators': 300, 'eta': 0.1, 'eval_metric': 'mlogloss', 'subsample': 0.8, 'min_child_weight': 6.0, 'max_depth': 4.0, 'silent': 1, 'nthread': 4, 'colsample_bytree': 0.8, 'objective': 'multi:softprob', 'num_class': 9}
	Score 0.5291524352341775



[299]	train-mlogloss:0.420914	eval-mlogloss:0.529153
Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.940268	eval-mlogloss:1.961795
[1]	train-mlogloss:1.748813	eval-mlogloss:1.785174
[2]	train-mlogloss:1.594283	eval-mlogloss:1.644938
[3]	train-mlogloss:1.465492	eval-mlogloss:1.528138
[4]	train-mlogloss:1.356044	eval-mlogloss:1.430730
[5]	train-mlogloss:1.261356	eval-mlogloss:1.346870
[6]	train-mlogloss:1.180560	eval-mlogloss:1.275888
[7]	train-mlogloss:1.107762	eval-mlogloss:1.212266
[8]	train-mlogloss:1.042772	eval-mlogloss:1.155850
[9]	train-mlogloss:0.985722	eval-mlogloss:1.106041
[10]	train-mlogloss:0.932568	eval-mlogloss:1.059688
[11]	train-mlogloss:0.883667	eval-mlogloss:1.018687
[12]	train-mlogloss:0.838580	eval-mlogloss:0.980013
[13]	train-mlogloss:0.797832	eval-mlogloss:0.945456
[14]	train-mlogloss:0.761621	eval-mlogloss:0.915087
[15]	train-mlogloss:0.727933	eval-mlogloss:0.886212
[16]	train-mlogloss:0.696873	eval-mlogloss:0.860327
[17]	train-mlo


Training with params : 
{'n_estimators': 300, 'eta': 0.1, 'eval_metric': 'mlogloss', 'subsample': 0.8, 'min_child_weight': 1.0, 'max_depth': 12.0, 'silent': 1, 'nthread': 4, 'colsample_bytree': 0.7000000000000001, 'objective': 'multi:softprob', 'num_class': 9}
	Score 0.4827659979108421



[166]	train-mlogloss:0.098293	eval-mlogloss:0.482766
Stopping. Best iteration:
[156]	train-mlogloss:0.106251	eval-mlogloss:0.482080

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:2.026212	eval-mlogloss:2.027853
[1]	train-mlogloss:1.922966	eval-mlogloss:1.926628
[2]	train-mlogloss:1.821144	eval-mlogloss:1.825369
[3]	train-mlogloss:1.737036	eval-mlogloss:1.741698
[4]	train-mlogloss:1.668966	eval-mlogloss:1.674527
[5]	train-mlogloss:1.608411	eval-mlogloss:1.614594
[6]	train-mlogloss:1.551491	eval-mlogloss:1.558452
[7]	train-mlogloss:1.505674	eval-mlogloss:1.513073
[8]	train-mlogloss:1.461077	eval-mlogloss:1.469139
[9]	train-mlogloss:1.423569	eval-mlogloss:1.431883
[10]	train-mlogloss:1.387483	eval-mlogloss:1.396514
[11]	train-mlogloss:1.356829	eval-mlogloss:1.366044
[12]	train-mlogloss:1.324248	eval-mlogloss:1.333932
[13]	train-mlogloss:1.294447	eval-mlogloss:1.304492
[14]	train-mlogloss:1.267025	eval-mlogloss:1.276814
[15]	train-mlogloss:1.241425	eval-mlog

...


Training with params : 
{'n_estimators': 300, 'eta': 0.1, 'eval_metric': 'mlogloss', 'subsample': 0.8, 'min_child_weight': 4.0, 'max_depth': 12.0, 'silent': 1, 'nthread': 4, 'colsample_bytree': 0.8, 'objective': 'multi:softprob', 'num_class': 9}
	Score 0.48020922559439383



[215]	train-mlogloss:0.124323	eval-mlogloss:0.480209
Stopping. Best iteration:
[205]	train-mlogloss:0.129957	eval-mlogloss:0.479997

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.951601	eval-mlogloss:1.968392
[1]	train-mlogloss:1.772841	eval-mlogloss:1.805750
[2]	train-mlogloss:1.629830	eval-mlogloss:1.674567
[3]	train-mlogloss:1.511651	eval-mlogloss:1.568577
[4]	train-mlogloss:1.402181	eval-mlogloss:1.471903
[5]	train-mlogloss:1.310399	eval-mlogloss:1.389996
[6]	train-mlogloss:1.228744	eval-mlogloss:1.319481
[7]	train-mlogloss:1.156228	eval-mlogloss:1.254892
[8]	train-mlogloss:1.090606	eval-mlogloss:1.197552
[9]	train-mlogloss:1.029369	eval-mlogloss:1.144238
[10]	train-mlogloss:0.976564	eval-mlogloss:1.098023
[11]	train-mlogloss:0.927614	eval-mlogloss:1.055682
[12]	train-mlogloss:0.883186	eval-mlogloss:1.017366
[13]	train-mlogloss:0.843564	eval-mlogloss:0.984009
[14]	train-mlogloss:0.807137	eval-mlogloss:0.953243
[15]	train-mlogloss:0.770710	eval-mlog


Training with params : 
{'n_estimators': 300, 'eta': 0.1, 'eval_metric': 'mlogloss', 'subsample': 0.8, 'min_child_weight': 1.0, 'max_depth': 12.0, 'silent': 1, 'nthread': 4, 'colsample_bytree': 0.5, 'objective': 'multi:softprob', 'num_class': 9}
	Score 0.48211445970646594



Stopping. Best iteration:
[173]	train-mlogloss:0.109998	eval-mlogloss:0.481827

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:2.054506	eval-mlogloss:2.056602
[1]	train-mlogloss:1.956810	eval-mlogloss:1.960148
[2]	train-mlogloss:1.843482	eval-mlogloss:1.848055
[3]	train-mlogloss:1.769828	eval-mlogloss:1.775484
[4]	train-mlogloss:1.696031	eval-mlogloss:1.702944
[5]	train-mlogloss:1.633584	eval-mlogloss:1.641336
[6]	train-mlogloss:1.571696	eval-mlogloss:1.579639
[7]	train-mlogloss:1.516580	eval-mlogloss:1.524739
[8]	train-mlogloss:1.470360	eval-mlogloss:1.479428
[9]	train-mlogloss:1.426807	eval-mlogloss:1.435765
[10]	train-mlogloss:1.386199	eval-mlogloss:1.395436
[11]	train-mlogloss:1.351775	eval-mlogloss:1.361011
[12]	train-mlogloss:1.318542	eval-mlogloss:1.327750
[13]	train-mlogloss:1.288165	eval-mlogloss:1.297568
[14]	train-mlogloss:1.262484	eval-mlogloss:1.272310
[15]	train-mlogloss:1.236517	eval-mlogloss:1.246537
[16]	train-mlogloss:1.215208	eval-mlogl


Training with params : 
{'n_estimators': 300, 'eta': 0.1, 'eval_metric': 'mlogloss', 'subsample': 0.8, 'min_child_weight': 4.0, 'max_depth': 2.0, 'silent': 1, 'nthread': 4, 'colsample_bytree': 0.7000000000000001, 'objective': 'multi:softprob', 'num_class': 9}
	Score 0.6149988265686674




In [26]:
trials.losses()

[0.5291524352341775,
 0.4827659979108421,
 0.6152931701463015,
 0.49393036772385845,
 0.47972127793712643,
 0.4775279421980562,
 0.4790733728739279,
 0.563702687025257,
 0.4927790839871725,
 0.5099130553830566,
 0.5636064294106335,
 0.4819818514799825,
 0.7336241891696162,
 0.4842176175953909,
 0.528053819195215,
 0.4764895292652049,
 0.48227183816013147,
 0.48020922559439383,
 0.48211445970646594,
 0.6149988265686674]

In [28]:
trials.argmin

{'colsample_bytree': 0.60000000000000009,
 'max_depth': 11.0,
 'min_child_weight': 5.0}